In [1]:
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from tradingview_screener import get_all_symbols
from tvDatafeed import TvDatafeed, Interval
from scipy.stats import linregress

def Down_Trend_Line(data, window=5):
    df = data.iloc[:-window].copy()
    hh_pairs = argrelextrema(df['close'].values, comparator=np.greater, order=window)[0]
    highest_point = df.iloc[hh_pairs].nlargest(1, 'close').sort_values(by='datetime')
    max_idx = data.index.get_loc(highest_point.index[0])
    df_next = df.iloc[max_idx:]

    hh_pairs_next = argrelextrema(df_next['close'].values, comparator=np.greater, order=window)[0]
    slopes = []
    intercepts = []

    # Iterate over each next highest point and calculate the slope
    for hh_next in hh_pairs_next:
        next_hp = df_next.iloc[hh_next]
        slope, intercept, _, _, _ = linregress([highest_point.index[0], next_hp.name], [highest_point['close'].values[0], next_hp['close']])
        # Store the slope
        slopes.append(slope)
        intercepts.append(intercept)

    slopes = [slope for slope in slopes if slope != 0]
    intercepts = [intercept for slope, intercept in zip(slopes, intercepts) if slope != 0]
    if slopes:
        # Find the minimum slope and its corresponding intercept
        min_slope_index = np.argmin(np.abs(slopes))
        min_slope = slopes[min_slope_index]
        min_intercept = intercepts[min_slope_index]
        df = data.copy()
        df['trend'] = min_slope * df.index + min_intercept
        df['Entry'] = df['close'] > df['trend']
    return df

tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)


Titles = ['Hisse Adı', 'Son Fiyat' ,'Trend Değeri','Yüzde', 'Yakınlık Durumu','Kırılma Durumu']
df_down_trend = pd.DataFrame(columns=Titles)
for i in range(0,len(Hisseler)):
    #print(Hisseler[i])
    try:
        order = 5
        data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=Interval.in_4_hour, n_bars=500)
        data = data.reset_index()
        Trend_line = Down_Trend_Line(data, window=order)
        Entry = False
        Signals = Trend_line.tail(order)
        Signals = Signals.reset_index()
        Last_Price = Signals.loc[order-1, 'close']
        Last_Trend = Signals.loc[order-1, 'close']
        Last_Perc = ((Signals.loc[order-1,'trend']- Signals.loc[order-1,'close'])/Signals.loc[order-1,'trend'])*100
        Close_Status = False
        Break_Status = False
        if Last_Perc<=5 and Last_Trend>Last_Price:
            Close_Status  = True

        Entry = Signals['Entry'].any() and not Signals['Entry'].all()
        Last_Entry = Signals.loc[order-1,'Entry']
        if Entry==True and Last_Entry==True:
              Break_Status = True

        L1 = [Hisseler[i] ,round(Last_Price,2), round(Last_Trend,2), round(Last_Perc,2),Close_Status,Break_Status]
        print(L1)
        df_down_trend.loc[len(df_down_trend)] = L1
    except:
        pass

df_close = df_down_trend[(df_down_trend['Yakınlık Durumu'] == True)]
df_Breakout = df_down_trend[(df_down_trend['Kırılma Durumu'] == True)]

print('Yakın Olan Hisseler')
print(df_close.to_string())
print('Kırılma Gerçekleşmiş Olanlar')
print(df_Breakout.to_string())

  Cloning https://github.com/rongardF/tvdatafeed to c:\users\serkan polat\appdata\local\temp\pip-req-build-4mudlb4z
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed 'C:\Users\Serkan POLAT\AppData\Local\Temp\pip-req-build-4mudlb4z'
you are using nologin method, data you access may be limited


['A1CAP', 28.58, 28.58, 30.18, False, False]
['ACSEL', 147.5, 147.5, 16.88, False, False]
['ADEL', 679.0, 679.0, 13.6, False, False]
['ADESE', 2.32, 2.32, 9.95, False, False]
['ADGYO', 28.42, 28.42, 28.31, False, False]
['AFYON', 13.43, 13.43, 11.55, False, False]
['AGROT', 18.81, 18.81, 46.26, False, False]
['AGYO', 8.52, 8.52, 11.49, False, False]
['AHGAZ', 16.49, 16.49, -15.44, False, False]
['AKBNK', 67.25, 67.25, -3.86, False, True]
['AKCNS', 142.7, 142.7, 5.85, False, False]
['AKENR', 16.37, 16.37, 16.02, False, False]
['AKFGY', 2.4, 2.4, 5.1, False, False]
['AKFYE', 27.2, 27.2, 1.93, False, False]
['AKGRT', 7.59, 7.59, 5.77, False, False]
['AKMGY', 304.5, 304.5, 18.41, False, False]
['AKSA', 118.0, 118.0, 2.15, False, False]
['AKSGY', 18.2, 18.2, -28.45, False, False]
['AKSUE', 13.72, 13.72, 13.6, False, False]
['ALARK', 118.1, 118.1, 1.6, False, False]
['ALCAR', 1294.0, 1294.0, 13.54, False, False]
['ALCTL', 114.3, 114.3, 10.98, False, False]
['ALFAS', 76.75, 76.75, 24.25, Fals

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


['GMTAS', 8.89, 8.89, 0.9, False, False]
['GOKNR', 21.42, 21.42, 17.09, False, False]
['GOLTS', 455.75, 455.75, 11.22, False, False]
['GOODY', 25.6, 25.6, 2.16, False, False]
['GOZDE', 25.8, 25.8, 15.04, False, False]
['GRNYO', 11.36, 11.36, -2.02, False, True]
['GRTRK', 67.6, 67.6, 12.19, False, False]
['GSDDE', 11.06, 11.06, 8.13, False, False]
['GSDHO', 4.33, 4.33, 7.87, False, False]
['GSRAY', 8.85, 8.85, -2.38, False, False]
['GUBRF', 162.5, 162.5, 2.8, False, False]
['GWIND', 30.16, 30.16, 0.49, False, False]
['GZNMI', 34.74, 34.74, 5.33, False, False]
['HATEK', 14.1, 14.1, 9.19, False, False]
['HATSN', 56.35, 56.35, 12.61, False, False]
['HDFGS', 2.01, 2.01, 12.04, False, False]
['HEDEF', 27.02, 27.02, 5.43, False, False]
['HEKTS', 15.28, 15.28, 20.65, False, False]
['HKTM', 19.09, 19.09, 10.35, False, False]
['HLGYO', 3.1, 3.1, 12.05, False, False]
['HTTBT', 70.75, 70.75, -3.77, False, False]
['HUBVC', 11.8, 11.8, -3.04, False, False]
['HUNER', 4.74, 4.74, 6.22, False, False]
[

...

['TMSN', 141.2, 141.2, 9.0, False, False]


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


['TRCAS', 29.78, 29.78, 3.42, False, False]
['TSGYO', 11.71, 11.71, 0.21, False, False]
['TSPOR', 2.09, 2.09, 7.57, False, False]
['TUCLK', 13.24, 13.24, 11.28, False, False]
['TUKAS', 7.75, 7.75, 11.64, False, False]
['TUPRS', 189.3, 189.3, 2.94, False, False]


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


['TURGG', 607.0, 607.0, 5.08, False, False]
['TURSG', 74.0, 74.0, 2.57, False, False]
['UFUK', 479.75, 479.75, 0.47, False, False]
['ULAS', 26.48, 26.48, 6.02, False, False]
['ULUFA', 13.92, 13.92, 0.82, False, False]
['ULUSE', 181.5, 181.5, 10.89, False, False]
['ULUUN', 27.02, 27.02, 17.5, False, False]
['UNLU', 15.62, 15.62, 8.37, False, False]
['USAK', 8.55, 8.55, 32.53, False, False]
['USDTR', 3013.0, 3013.0, 1.48, False, False]
['UZERB', 700.5, 700.5, 20.9, False, False]
['VAKFN', 3.58, 3.58, -2.33, False, True]
['VANGD', 20.36, 20.36, 18.27, False, False]
['VBTYZ', 30.22, 30.22, 7.44, False, False]
['VERTU', 45.0, 45.0, -2.2, False, False]
['VERUS', 262.0, 262.0, 5.38, False, False]
['VESBE', 22.1, 22.1, 3.51, False, False]
['VESTL', 89.9, 89.9, 7.98, False, False]
['VKFYO', 27.1, 27.1, -2.24, False, True]
['VKGYO', 2.45, 2.45, 1.93, False, False]
['VKING', 43.96, 43.96, 8.0, False, False]
['VRGYO', 25.62, 25.62, 2.68, False, False]
['YATAS', 38.58, 38.58, 2.21, False, False]
['

In [2]:
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener
import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from tradingview_screener import get_all_symbols
from tvDatafeed import TvDatafeed, Interval
from scipy.stats import linregress

def Down_Trend_Line(data, window=5):
    df = data.iloc[:-window].copy()
    hh_pairs = argrelextrema(df['close'].values, comparator=np.greater, order=window)[0]
    highest_point = df.iloc[hh_pairs].nlargest(1, 'close').sort_values(by='datetime')
    max_idx = data.index.get_loc(highest_point.index[0])
    df_next = df.iloc[max_idx:]

    hh_pairs_next = argrelextrema(df_next['close'].values, comparator=np.greater, order=window)[0]
    slopes = []
    intercepts = []

    # Iterate over each next highest point and calculate the slope
    for hh_next in hh_pairs_next:
        next_hp = df_next.iloc[hh_next]
        slope, intercept, _, _, _ = linregress([highest_point.index[0], next_hp.name], [highest_point['close'].values[0], next_hp['close']])
        # Store the slope
        slopes.append(slope)
        intercepts.append(intercept)

    slopes = [slope for slope in slopes if slope != 0]
    intercepts = [intercept for slope, intercept in zip(slopes, intercepts) if slope != 0]
    if slopes:
        # Find the minimum slope and its corresponding intercept
        min_slope_index = np.argmin(np.abs(slopes))
        min_slope = slopes[min_slope_index]
        min_intercept = intercepts[min_slope_index]
        df = data.copy()
        df['trend'] = min_slope * df.index + min_intercept
        df['Entry'] = df['close'] > df['trend']
    return df

tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)


Titles = ['Hisse Adı', 'Son Fiyat' ,'Trend Değeri','Yüzde', 'Yakınlık Durumu','Kırılma Durumu']
df_down_trend = pd.DataFrame(columns=Titles)
for i in range(0,len(Hisseler)):
    #print(Hisseler[i])
    try:
        order = 5
        data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=Interval.in_4_hour, n_bars=500)
        data = data.reset_index()
        Trend_line = Down_Trend_Line(data, window=order)
        Entry = False
        Signals = Trend_line.tail(order)
        Signals = Signals.reset_index()
        Last_Price = Signals.loc[order-1, 'close']
        Last_Trend = Signals.loc[order-1, 'close']
        Last_Perc = ((Signals.loc[order-1,'trend']- Signals.loc[order-1,'close'])/Signals.loc[order-1,'trend'])*100
        Close_Status = False
        Break_Status = False
        if Last_Perc<=5 and Last_Trend>Last_Price:
            Close_Status  = True

        Entry = Signals['Entry'].any() and not Signals['Entry'].all()
        Last_Entry = Signals.loc[order-1,'Entry']
        if Entry==True and Last_Entry==True:
              Break_Status = True

        L1 = [Hisseler[i] ,round(Last_Price,2), round(Last_Trend,2), round(Last_Perc,2),Close_Status,Break_Status]
        print(L1)
        df_down_trend.loc[len(df_down_trend)] = L1
    except:
        pass

df_close = df_down_trend[(df_down_trend['Yakınlık Durumu'] == True)]
df_Breakout = df_down_trend[(df_down_trend['Kırılma Durumu'] == True)]

print('Yakın Olan Hisseler')
print(df_close.to_string())
print('Kırılma Gerçekleşmiş Olanlar')
print(df_Breakout.to_string())

  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed 'C:\Users\Serkan POLAT\AppData\Local\Temp\pip-req-build-5wbz10hq'


  Cloning https://github.com/rongardF/tvdatafeed to c:\users\serkan polat\appdata\local\temp\pip-req-build-5wbz10hq
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
['A1CAP', 28.4, 28.4, 30.62, False, False]
['ACSEL', 147.4, 147.4, 16.94, False, False]
['ADEL', 674.5, 674.5, 14.18, False, False]
['ADESE', 2.31, 2.31, 10.34, False, False]
['ADGYO', 28.72, 28.72, 27.55, False, False]
['AFYON', 13.4, 13.4, 11.74, False, False]
['AGROT', 18.81, 18.81, 46.26, False, False]
['AGYO', 8.53, 8.53, 11.38, False, False]
['AHGAZ', 16.5, 16.5, -15.51, False, False]
['AKBNK', 68.0, 68.0, -5.02, False, True]
['AKCNS', 142.0, 142.0, 6.31, False, False]
['AKENR', 16.41, 16.41, 15.81, False, False]
['AKFGY', 2.4, 2.4, 5.1, False, False]
['AKFYE', 27.06, 27.06, 2.43, False, False]
['AKGRT', 7.59, 7.59, 5.77, False, False]
['AKMGY', 304.5, 304.5, 18.41, 

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


['HEKTS', 15.33, 15.33, 20.39, False, False]
['HKTM', 19.4, 19.4, 8.89, False, False]
['HLGYO', 3.09, 3.09, 12.33, False, False]
['HTTBT', 70.05, 70.05, -2.74, False, False]
['HUBVC', 11.78, 11.78, -2.87, False, False]
['HUNER', 4.7, 4.7, 7.01, False, False]
['HURGZ', 5.6, 5.6, 5.2, False, False]
['ICBCT', 15.87, 15.87, 12.67, False, False]
['ICUGS', 13.66, 13.66, 19.6, False, False]
['IEYHO', 4.9, 4.9, 7.32, False, False]
['IHAAS', 21.62, 21.62, -17.42, False, False]
['IHEVA', 3.64, 3.64, -180.4, False, False]
['IHGZT', 1.24, 1.24, 12.93, False, False]
['IHLGM', 1.56, 1.56, 17.66, False, False]
['IHYAY', 3.22, 3.22, 15.89, False, False]
['IMASM', 14.38, 14.38, -7.31, False, False]
['INDES', 9.05, 9.05, 9.56, False, False]
['INFO', 12.44, 12.44, 33.62, False, False]
['INGRM', 551.0, 551.0, 6.03, False, False]
['INTEM', 265.5, 265.5, 13.14, False, False]
['INVEO', 43.86, 43.86, 15.87, False, False]
['INVES', 339.5, 339.5, 5.79, False, False]
['IPEKE', 41.2, 41.2, 6.8, False, False]
['IS

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


...

['SAFKR', 40.24, 40.24, 8.97, False, False]
['SAMAT', 42.1, 42.1, 3.44, False, False]
['SANEL', 29.88, 29.88, 5.0, False, False]
['SANFM', 59.8, 59.8, 15.77, False, False]
['SANKO', 27.76, 27.76, 6.55, False, False]
['SARKY', 34.38, 34.38, 14.86, False, False]
['SASA', 47.46, 47.46, 4.72, False, False]
['SAYAS', 70.65, 70.65, 26.9, False, False]
['SDTTR', 286.25, 286.25, 24.48, False, False]
['SEGYO', 4.5, 4.5, 32.82, False, False]
['SEKFK', 10.55, 10.55, 7.99, False, False]
['SEKUR', 14.23, 14.23, 14.63, False, False]
['SELEC', 51.9, 51.9, 19.84, False, False]
['SELGD', 48.4, 48.4, 23.22, False, False]
['SELVA', 15.55, 15.55, 4.54, False, False]
['SISE', 54.2, 54.2, 2.36, False, False]
['SKBNK', 5.13, 5.13, 8.42, False, False]
['SKTAS', 5.75, 5.75, 6.77, False, False]
['SKYLP', 127.4, 127.4, 12.96, False, False]
['SKYMD', 12.04, 12.04, -6.66, False, False]
['SMART', 40.26, 40.26, -2.55, False, False]
['SMRTG', 54.2, 54.2, 21.86, False, False]
['SNICA', 7.9, 7.9, 11.66, False, False]
[

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


['TUCLK', 13.24, 13.24, 11.28, False, False]
['TUKAS', 7.75, 7.75, 11.64, False, False]
['TUPRS', 189.3, 189.3, 2.94, False, False]
['TURGG', 607.0, 607.0, 5.08, False, False]
['TURSG', 74.0, 74.0, 2.57, False, False]
['UFUK', 479.75, 479.75, 0.47, False, False]
['ULAS', 26.48, 26.48, 6.02, False, False]
['ULUFA', 13.92, 13.92, 0.82, False, False]
['ULUSE', 181.5, 181.5, 10.89, False, False]
['ULUUN', 27.02, 27.02, 17.5, False, False]
['UNLU', 15.62, 15.62, 8.37, False, False]
['USAK', 8.55, 8.55, 32.53, False, False]
['USDTR', 3013.0, 3013.0, 1.48, False, False]
['UZERB', 700.5, 700.5, 20.9, False, False]


ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


['VANGD', 20.36, 20.36, 18.27, False, False]
['VBTYZ', 30.22, 30.22, 7.44, False, False]
['VERTU', 45.0, 45.0, -2.2, False, False]
['VERUS', 262.0, 262.0, 5.38, False, False]
['VESBE', 22.1, 22.1, 3.51, False, False]
['VESTL', 89.9, 89.9, 7.98, False, False]
['VKFYO', 27.1, 27.1, -2.24, False, True]
['VKGYO', 2.45, 2.45, 1.93, False, False]
['VKING', 43.96, 43.96, 8.0, False, False]
['VRGYO', 25.62, 25.62, 2.68, False, False]
['YATAS', 38.58, 38.58, 2.21, False, False]
['YAYLA', 18.42, 18.42, -9.43, False, False]
['YEOTK', 190.6, 190.6, 14.04, False, False]
['YESIL', 4.55, 4.55, 14.06, False, False]
['YKSLN', 22.0, 22.0, -16.33, False, False]
['YONGA', 86.1, 86.1, 17.98, False, False]
['YUNSA', 98.85, 98.85, -1.04, False, True]
['YYLGD', 13.04, 13.04, 10.19, False, False]
['Z30EA', 138.5, 138.5, 2.37, False, False]
['Z30KE', 104.7, 104.7, 1.23, False, False]
['Z30KP', 149.05, 149.05, 0.83, False, False]
['ZGOLD', 256.1, 256.1, 1.66, False, False]
['ZOREN', 6.09, 6.09, 8.53, False, Fals